# Why are learned indexes so effective?

This notebook can be used to reproduce the figures and the tables in the papers

> Paolo Ferragina, Fabrizio Lillo, and Giorgio Vinciguerra. _On the performance of learned data structures_. Theoretical Computer Science, 2021.

> Paolo Ferragina, Fabrizio Lillo, and Giorgio Vinciguerra. _Why are learned indexes so effective?_. In Proceedings of the 37th International Conference on Machine Learning (ICML). PMLR, 2020.


In [ ]:
!pip install matplotlib>=3.0.3
!pip install tikzplotlib==0.9.0

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib
import tikzplotlib
import matplotlib.pyplot as plt

# Figure 3

The following two cells generate Figure 3 of the paper, containing theoretical and empirical mean exit times for several distributions and algorithms. The script to generate the raw data is `run_main.sh`.

In [ ]:
path = './results/'
files = ['uniform_0_1_met3.csv', 'uniform_0_10_met3.csv', 'uniform_10_100_met4.481.csv',
         'pareto_scale2_shape2.5_met1.25.csv', 'pareto_scale3_shape3_met3.csv', 'pareto_scale4_shape3.5_met5.25.csv',
         'lognormal_m1_s1_met0.582.csv', 'lognormal_m1_s0.75_met1.324.csv', 'lognormal_m1_s0.5_met3.521.csv',
         'gamma_scale1_shape1_met1.csv', 'gamma_scale3_shape2_met2.csv', 'gamma_scale6_shape3_met3.csv']
dfs = [(f, pd.read_csv(path + f, comment='#')) for f in files]

def label_params(l):
    if l[0] == 'uniform':
        return 'Uniform $a=%s,b=%s$' % (l[1], l[2])
    if l[0] == 'pareto':
        return 'Pareto $k=%s,\\alpha=%s$' % (l[1][5:], l[2][5:])
    if l[0] == 'lognormal':
        return 'Lognormal $\\mu=%s,\\sigma=%s$' % (l[1][1:], l[2][1:])
    if l[0] == 'gamma':
        return 'Gamma $\\theta=%s,k=%s$' % (l[1][5:], l[2][5:])

In [ ]:
_, axes = plt.subplots(nrows=4, ncols=3, sharex=True, sharey='row', figsize=(3 * 6.75, 3.4 * 6.75))

for i, (f, d), ax in zip(range(len(dfs)), dfs, axes.flatten()):
    if not f.startswith(('uniform', 'pareto', 'lognormal', 'exponential', 'gamma')):
        continue
    
    step = 2
    x = d.epsilon[::step]
    
    fmt = {'markersize': 1}
    ax.plot(x, d.opt_avg[::step], '.C2', **fmt, label='OPT')
    ax.plot(x, d.met_avg[::step], '.C3',**fmt, label='MET')
    
    fmt = {'linestyle': (0, (5, 1)), 'alpha': 0.3}
    ax.fill_between(x, d.opt_avg[::step]-d.opt_std[::step], d.opt_avg[::step]+d.opt_std[::step], **fmt, facecolor='C2')
    ax.fill_between(x, d.met_avg[::step]-d.met_std[::step], d.met_avg[::step]+d.met_std[::step], **fmt, facecolor='C3')
    
    met_constant = float(f.split('_')[-1][3:-4])
    met = met_constant * d.epsilon ** 2
    ax.plot(x, met[::step], 'k-', label=r'Thm 1 ($%s\varepsilon^2$)' % met_constant, linewidth=0.5)
    ax.plot(x, met[::step] * (1 + np.sqrt(2/3)), 'k--', label='_nolegend_', linewidth=0.5)
    ax.plot(x, met[::step] * (1 - np.sqrt(2/3)), 'k--', label='_nolegend_', linewidth=0.5)
    
    if i % 3 == 0: ax.set_ylabel('Mean segment length $\\cdot 10^6$')
    if i > 8: ax.set_xlabel(r'$\varepsilon$')
    ax.set_xlim(-5, 261)
    ax.legend(loc='upper left')
    ax.set_title(label_params(f.split('_')[0:3]))
    ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

tikz = tikzplotlib.get_tikz_code(
    figurewidth='2.46375in',
    figureheight='2.13in',
    standalone=True,
    float_format='{:.4f}',
    extra_tikzpicture_parameters=['/pgfplots/group/.cd', 'xlabels at=edge bottom', 'ylabels at=edge left', 'yticklabels at=edge left', 'vertical sep=36pt', 'horizontal sep=12pt', 'xticklabels at=edge bottom', '/pgfplots/.cd', 'legend image post style={mark size=1.5pt}', 'ytick scale label code/.code={}'],
    extra_axis_parameters=['xtick={0,50,...,250}', 'tick align=outside', 'tickpos=left', 'minor tick num=4', 'ylabel style={at={(-0.18,0.5)}}', 'title style={at={(0.5,1)}, yshift=-1ex,anchor=south, align=center}', 'every y tick scale label/.style={at={(yticklabel* cs:1)}, xshift=0.5ex, anchor=south}', 'legend style={nodes={scale=0.7, transform shape}}', 'clip marker paths=true', '/pgf/number format/1000 sep={}']
)

with open('experiments.tex', 'w') as f:
    f.write(tikz
            .replace('mark size=1,', 'mark size=0.15,')
            .replace(r'{Gamma \(\displaystyle \theta=1,k=1\)}', r'{Gamma $\theta=1, k=1$ \\ {\footnotesize (Exponential $\lambda=1$)}}'))

# Table 1

The following cell generates Table 1 of the paper, containing the average slope found by the optimal PLA algorithm. The script to generate the raw data is `run_main.sh`, which is the one ran for the previous figure, so there is need to run it again.

In [ ]:
out = pd.DataFrame(columns=['Distribution', 'Slope lo', 'Slope hi'])
for f, df in dfs:
    if not f.startswith(('uniform', 'pareto', 'lognormal', 'exponential', 'gamma')):
        continue
        
    i = 0 if np.isnan(out.index.max()) else out.index.max() + 1
    split = " ".join(f.split('_')[0:3]).capitalize()
    out.loc[i] = [label_params(f.split('_')[0:3]), df.opt_lo_avg.mean(), df.opt_hi_avg.mean()]

out

# Figure 4

The following two cells generate Figure 4 of the paper. The script to generate the raw data is `run_assumption_tests.sh`.

In [ ]:
filenames = filter(lambda f: f.endswith('.csv') and f.startswith('assum'), os.listdir(path))
dfs2 = [(f, pd.read_csv(path + f, comment='#')) for f in filenames]

In [ ]:
sigma_mu_ratios = ['0.15', '1.5', '15']
_, axes = plt.subplots(nrows=1, ncols=3, figsize=(17, 4.5))

def pick_color(f):
    if 'gamma' in f:
        return plt.cm.Dark2(4)
    if 'lognormal' in f:
        return plt.cm.Dark2(6)
    if 'uniform' in f:
        return plt.cm.Dark2(5)
    if 'pareto' in f:
        return plt.cm.Dark2(2)

for (i, ratio), ax in zip(enumerate(sigma_mu_ratios), axes.flatten()):
    dfs_filtered = list(filter(lambda o: o[0].startswith('assum%s_' % ratio), dfs2))
    constant = float(ratio) ** -2
    
    for f, df in dfs_filtered:
        theoretical_met = constant * df.epsilon ** 2 
        label = label_params(f[:-4].split('_')[1:4])
        ax.set_title(r'$\sigma/\mu=%s$' % ratio)
        ax.plot(df.epsilon, (df.met_avg - theoretical_met) / df.met_avg, '.',  markersize=2, label=label, c=pick_color(f))
    
    ax.axhline(0, c='k', label=r'$%.3f\varepsilon^2$' % constant)
    if i == 0: ax.set_ylabel('Relative error')
    ax.set_xlabel(r'$\varepsilon$')
    ax.legend()
    ax.ticklabel_format(style='plain', axis='y', scilimits=(0,0))

tikz = tikzplotlib.get_tikz_code(
    figurewidth='2.425in',
    figureheight='2.13in',
    standalone=True,
    float_format='{:.4f}',
    extra_tikzpicture_parameters=['/pgfplots/group/.cd', 'horizontal sep=25pt', '/pgfplots/.cd', 'legend image post style={mark size=2pt}', 'yticklabel style={/pgf/number format/.cd,fixed,1000 sep={}}'],
    extra_axis_parameters=['xtick={0,50,...,250}', 'scaled y ticks = false', 'tick align=outside', 'tickpos=left', 'minor tick num=4', 'ylabel style={at={(-0.20,0.5)}}', 'title style={at={(0.5,1)}, yshift=-1ex,anchor=south, align=center}', 'every y tick scale label/.style={at={(yticklabel* cs:1)}, xshift=0.5ex, anchor=south}', 'legend style={nodes={scale=0.65, transform shape}}', 'clip marker paths=true']
)
with open('assumptions.tex', 'w') as f:
    f.write(tikz.replace('mark size=1,', 'mark size=0.15,'))

# Figure 5

The following two cells generate Figure 5 of the paper. The script to generate the raw data is `run_segments_count.sh`.

In [ ]:
filenames = sorted(filter(lambda f: f.endswith('.csv') and f.startswith('count'), os.listdir(path)), reverse=True)
dfs3 = [(f[:-4], pd.read_csv(path + f, comment='#')) for f in filenames]

In [ ]:
for i, (f, d) in enumerate(dfs3):
    df = pd.concat([d[1:100], d[100::50]])
    color = pick_color(f)
    met = float(f.split('_')[4][3:])
    epsilon = float(f.split('_')[5][7:])
    theoretical = 1 / (met * epsilon ** 2)
    plt.plot(df.n, df.segments_avg / df.n, c=color, label=label_params(f.split('_')[1:4]))
    plt.axhline(y=theoretical, color=color, linestyle='--', linewidth=0.5, label='$\lambda=%.6f$' % theoretical)
    plt.fill_between(df.n, (df.segments_avg + df.segments_std) / df.n, (df.segments_avg - df.segments_std) / df.n, alpha=0.3, facecolor=color)

plt.xscale('log')
plt.xlabel('$n$')
plt.ylabel(r'$(s/n)$')
plt.legend(loc='upper right')

tikz = tikzplotlib.get_tikz_code(
    standalone=True,
    figurewidth='234.8775pt',
    figureheight='190pt',
    float_format='{:.8f}',
    extra_tikzpicture_parameters=['/pgfplots/.cd', 'compat=1.3', r'tick scale labels in axis labels/.code={\pgfkeysgetvalue{/pgfplots/ytick scale label code/.@cmd}\temp\pgfkeyslet{/pgfplots/ytick scale label code orig/.@cmd}\temp\pgfkeysalso{ytick scale label code/.code={\gdef\yTickScale{##1}}, every axis/.append style={tick scale binop={}, ylabel/.add = {}{$\cdot$\pgfplotsset{ytick scale label code orig=\yTickScale}}}}}'],
    extra_axis_parameters=['tick scale labels in axis labels', 'tick align=outside', 'tickpos=left', 'minor tick num=4', 'legend style={nodes={scale=0.65, transform shape}}', 'clip marker paths=true']
)
with open('convergence.tex', 'w') as f:
    f.write(tikz)
plt.show()

# Figure 6

The following cell generates Figure 6 of the paper. The script to generate the raw data is `run_real_gaps.sh`.

In [ ]:
from scipy.optimize import curve_fit

df4 = pd.read_csv('results/real_gaps.csv')

def quadratic(x, a, b):
    return a * x ** b

cm = plt.cm.Paired
ci = 2

for name in df4.dataset.unique():
    df = df4[df4.dataset == name]
    name = name.split('_')[0]
    (a, b), _ = curve_fit(quadratic, df.epsilon, df.opt_avg)
    plt.plot(df.epsilon, df.opt_avg, '.', label='OPT on ' + name, markersize=1, color=cm(ci))
    plt.plot(df.epsilon, quadratic(df.epsilon, a, b), '--', color=cm(ci + 1), label=r'Fitted $%.3f \, \varepsilon^{%.3f}$ on %s' % (a, b, name))
    ci += 2
    
plt.xlabel('$\\varepsilon$')
plt.ylabel('Mean segmenth length ')
plt.legend(loc='upper left')

tikz = tikzplotlib.get_tikz_code(
    standalone=True,
    figurewidth='234.8775pt',
    figureheight='190pt',
    float_format='{:.8f}',
    extra_tikzpicture_parameters=['/pgfplots/.cd', 'compat=1.3', 'legend image post style={mark size=1.5pt}', r'tick scale labels in axis labels/.code={\pgfkeysgetvalue{/pgfplots/ytick scale label code/.@cmd}\temp\pgfkeyslet{/pgfplots/ytick scale label code orig/.@cmd}\temp\pgfkeysalso{ytick scale label code/.code={\gdef\yTickScale{##1}}, every axis/.append style={tick scale binop={}, ylabel/.add = {}{$\,\cdot$\pgfplotsset{ytick scale label code orig=\yTickScale}}}}}'],
    extra_axis_parameters=['tick scale labels in axis labels', 'tick align=outside', 'tickpos=left', 'minor tick num=4', 'legend style={nodes={scale=0.65, transform shape}}', 'clip marker paths=true']
).replace('mark size=1,', 'mark size=0.15,')
with open('real.tex', 'w') as f:
    f.write(tikz)
plt.show()

# Figure 7

The following cell generates Figure 7 of the paper. The script to generate the raw data is `run_main.sh`, which is the one ran for Figure 3, so there is need to run it again.

In [ ]:
_, axes = plt.subplots(nrows=1, ncols=3, sharex=True, sharey='row', figsize=(17, 4.3))
ma_orders = ['5', '50', '500']

for i, (ma_order, ax) in enumerate(zip(ma_orders, axes.flatten())):
    d = pd.read_csv('results/ma%s.csv' % ma_order, comment='#')
    step = 2
    x = d.epsilon[::step]

    fmt = {'markersize': 1}
    ax.plot(x, d.opt_avg[::step], '.C2', **fmt, label='OPT')
    ax.plot(x, d.met_avg[::step], '.C3',**fmt, label='MET')

    fmt = {'linestyle': (0, (5, 1)), 'alpha': 0.3}
    ax.fill_between(x, d.opt_avg[::step]-d.opt_std[::step], d.opt_avg[::step]+d.opt_std[::step], **fmt, facecolor='C2')
    ax.fill_between(x, d.met_avg[::step]-d.met_std[::step], d.met_avg[::step]+d.met_std[::step], **fmt, facecolor='C3')

    met = 3 * d.epsilon ** 2
    ax.plot(x, met[::step], 'k-', label=r'Equation 7', linewidth=0.5)
    ax.ticklabel_format(style='plain', axis='y', scilimits=(0,0))
    ax.legend(loc='upper left')

    if i == 0: ax.set_ylabel('Mean segment length $\\cdot 10^6$')
    ax.set_xlabel(r'$\varepsilon$')
    ax.set_title(r'$\ell_0=%s$' % ma_order)

tikz = tikzplotlib.get_tikz_code(
    figurewidth='2.425in',
    figureheight='2.13in',
    standalone=True,
    float_format='{:.4f}',
    extra_tikzpicture_parameters=['/pgfplots/group/.cd', 'xlabels at=edge bottom', 'ylabels at=edge left', 'yticklabels at=edge left', 'vertical sep=36pt', 'horizontal sep=12pt', 'xticklabels at=edge bottom', '/pgfplots/.cd', 'legend image post style={mark size=1.5pt}', 'ytick scale label code/.code={}'],
    extra_axis_parameters=['xtick={0,50,...,250}', 'tick align=outside', 'tickpos=left', 'minor tick num=4', 'ylabel style={at={(-0.18,0.5)}}', 'title style={at={(0.5,1)}, yshift=-1ex,anchor=south, align=center}', 'every y tick scale label/.style={at={(yticklabel* cs:1)}, xshift=0.5ex, anchor=south}', 'legend style={nodes={scale=0.7, transform shape}}', 'clip marker paths=true', '/pgf/number format/1000 sep={}']
)
with open('moving-average.tex', 'w') as f:
    f.write(tikz.replace('mark size=1,', 'mark size=0.15,'))

# Figure 8

The following cell generates Figure 8 of the paper. The script to generate the raw data is `run_main.sh`, which is the one ran for Figure 3, so there is need to run it again.

In [ ]:
_, axes = plt.subplots(nrows=1, ncols=3, sharex=True, sharey='row', figsize=(17, 4.3))
files = ['ar0.1.csv', 'ar0.6.csv', 'ar0.9.csv']
phi_values = ['0.1', '0.5', '0.9']
met_consts = [3.0000000000000004, 3, 2.9999999999999996]

for i, (phi_value, met_const, ax) in enumerate(zip(phi_values, met_consts, axes.flatten())):
    d = pd.read_csv('results/ar%s.csv' % phi_value, comment='#')
    step = 2
    x = d.epsilon[::step]

    fmt = {'markersize': 1}
    ax.plot(x, d.opt_avg[::step], '.C2', **fmt, label='OPT')
    ax.plot(x, d.met_avg[::step], '.C3',**fmt, label='MET')

    fmt = {'linestyle': (0, (5, 1)), 'alpha': 0.3}
    ax.fill_between(x, d.opt_avg[::step]-d.opt_std[::step], d.opt_avg[::step]+d.opt_std[::step], **fmt, facecolor='C2')
    ax.fill_between(x, d.met_avg[::step]-d.met_std[::step], d.met_avg[::step]+d.met_std[::step], **fmt, facecolor='C3')

    met = 3 * d.epsilon ** 2
    ax.plot(x, met[::step], 'k-', label=r'Equation 8', linewidth=0.5)
    ax.ticklabel_format(style='plain', axis='y', scilimits=(0,0))
    ax.legend(loc='upper left')

    if i == 0: ax.set_ylabel('Mean segment length $\\cdot 10^6$')
    ax.set_xlabel(r'$\varepsilon$')
    ax.set_title(r'$\varphi=%s$' % phi_value)

tikz = tikzplotlib.get_tikz_code(
    figurewidth='2.425in',
    figureheight='2.13in',
    standalone=True,
    float_format='{:.4f}',
    extra_tikzpicture_parameters=['/pgfplots/group/.cd', 'xlabels at=edge bottom', 'ylabels at=edge left', 'yticklabels at=edge left', 'vertical sep=36pt', 'horizontal sep=12pt', 'xticklabels at=edge bottom', '/pgfplots/.cd', 'legend image post style={mark size=1.5pt}', 'ytick scale label code/.code={}'],
    extra_axis_parameters=['xtick={0,50,...,250}', 'tick align=outside', 'tickpos=left', 'minor tick num=4', 'ylabel style={at={(-0.18,0.5)}}', 'title style={at={(0.5,1)}, yshift=-1ex,anchor=south, align=center}', 'every y tick scale label/.style={at={(yticklabel* cs:1)}, xshift=0.5ex, anchor=south}', 'legend style={nodes={scale=0.7, transform shape}}', 'clip marker paths=true', '/pgf/number format/1000 sep={}']
)
with open('autoregressive.tex', 'w') as f:
    f.write(tikz.replace('mark size=1,', 'mark size=0.15,'))